In [28]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
pd.options.display.max_colwidth = 100

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

# Non-DataScience/ML packages
import chardet
import re
import string

In [29]:
# Test your model with different holdout groups.
def run_with_the_holdouts(fit_type, data, target, holdout_percent):
    from sklearn.model_selection import train_test_split
    # Use train_test_split to create the necessary training and test groups
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=holdout_percent, random_state=20)
    print('With {0:.0%} Holdout: '.format(holdout_percent*1) + str(fit_type.fit(X_train, y_train).score(X_test, y_test)))
    print('Testing on Sample: ' + str(fit_type.fit(data, target).score(data, target)))

In [30]:
def run_the_cross_validations(fit_type, data, target):
    from sklearn.model_selection import cross_val_score
    return cross_val_score(bnb, data, target, cv=10)

In [31]:
def find_suitable_variables():
    num_columns3 = num_columns
    cols3_orig = ['LarcenyTheft']
#     num_columns3.remove('Population')
    num_columns2 = ['PropertyCrime']
    max_r_squared = 0
    max_xy_pair = []
    max_xy_pair = []
    for Y_variable in num_columns2:
        if Y_variable != 'Population':
            Y = df[Y_variable].values.reshape(-1,1)
            print("Y variable is {}".format(Y_variable))
            for col in num_columns:
                cols3 = cols3_orig
                if col != Y_variable:
                    cols3.append(col)
                    print("Now calculating with X = {}, and Y_variable={}".format(cols3, Y_variable))
                    X = df[[cols3]]
                    regr.fit(X, Y)
                    # Inspect the results.
                    print('\nCoefficients: \n', regr.coef_)
                    print('\nIntercept: \n', regr.intercept_)
                    print('\nR-squared:')
                    rs = regr.score(X,Y)
#                     print("rs is a {} type variable".format(type(rs)))
#                     if max_r_squared == 0 or rs > max_r_squared:
#                         max_r_squared = rs.astype(type('float',(float,),{}))
#                         max_xy_pair[0] = Y_variable
#                         max_xy_pair[1] = col
                    print(regr.score(X, Y))
                    X = df[['Robbery','Burglary','Murder','Rape','LarcenyTheft','Arson']]
#     print("Golden Y variable={}, and X variable={}, and max_r_squared={}".format(max_xy_pair[0],
#                                                                              max_xy_pair[1],
#                                                                              max_r_squared))
#     return max_xy_pair, max_r_squared

In [32]:
path="../../../../Datafiles/"
file = 'table_8_offenses_known_to_law_enforcement_california_by_city_2013.xls'
df = pd.read_excel(path + file, header=4)
print("there are {} entries in the data frame".format(len(df)))

there are 464 entries in the data frame


In [33]:
# Let's clean up the column names
colnames = df.columns
newcolnameslist = []

for colname in colnames:
    newcolname=colname.replace('\n','').replace(' ','').capitalize() # strip out newlines, spaces, and captitalize
    newcolname=re.sub(r'\([^)]*\)', '', newcolname)                  # remove parenthesized stuff
    newcolnameslist.append(newcolname)                               # put all the column names into a list 

df.columns = newcolnameslist
print(newcolnameslist)

['City', 'Population', 'Violentcrime', 'Murderandnonnegligentmanslaughter', 'Rape1', 'Rape2', 'Robbery', 'Aggravatedassault', 'Propertycrime', 'Burglary', 'Larceny-theft', 'Motorvehicletheft', 'Arson']


In [34]:
def return_diff_min_max_cv_score(cv_in):
    print("min cv_score={0:.4%}, max={1:.4%}, mean={3:.4%}, and delta is {2:.4%}".format(cv_in.min(),
                                                                      cv_in.max(), 
                                                                      cv_in.max()-cv_in.min(),
                                                                      cv_in.mean()))

In [35]:
df[df['City'].str.contains('figures shown')]
df[df['City'].str.contains('publish')]

df = df[(df['City'].str.contains('figures shown') == False)] # We don't want any documentation rows
df = df[(df['City'].str.contains('publish') == False)]       # We don't want any other documentation rows
# df = df[(df['City'].str.contains('Los Angeles') == False)]       # Remove the outlier

# All Rape1 values are false
# No Rape2 values are true
# Arson3 NaN's -> 161
# There are 161 NaN entries for Arson3
# Remove Los Angeles -- it's an outlier.

In [36]:
df['PopulationSquared'] = df.Population.pow(2)

df['Robbery'] = np.where(df['Robbery']>=1, 1, 0.0)
df = df.rename(columns={'Arson3': 'Arson', 'Rape2': 'Rape', 'Aggravatedassault': 'AggravatedAssault',
                        'Violentcrime':'ViolentCrime','Propertycrime': 'PropertyCrime', 
                        'Larceny-theft':'LarcenyTheft','Motorvehicletheft':'MotorVehicleTheft'})
# df.Arson = df.Arson.astype(int) # Let's make it an int
df['Murder'] = np.where(df['Murderandnonnegligentmanslaughter']>=1, 1, 0.0)
df.Arson.fillna(0, inplace=True) # Let's clean up Arson3

In [37]:
factor=100000
df_pop=df
df['ViolentCrime_pop'] = df['ViolentCrime']/df['Population']*factor
df['Rape_pop'] = df['Rape']/df['Population']*factor
df['AggravatedAssault_pop'] = df['AggravatedAssault']/df['Population']*factor
df['PropertyCrime_pop'] = df['PropertyCrime']/df['Population']*factor
df['Burglary_pop'] = df['Burglary']/df['Population']*factor
df['LarcenyTheft_pop'] = df['LarcenyTheft']/df['Population']*factor
df['MotorVehicleTheft_pop'] = df['MotorVehicleTheft']/df['Population']*factor
df['Arson_pop'] = df['Arson']/df['Population']*factor

In [38]:
df.head(5)
# df.describe()

,City,Population,ViolentCrime,Murderandnonnegligentmanslaughter,Rape1,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,...,PopulationSquared,Murder,ViolentCrime_pop,Rape_pop,AggravatedAssault_pop,PropertyCrime_pop,Burglary_pop,LarcenyTheft_pop,MotorVehicleTheft_pop,Arson_pop
0,Adelanto,31165.000,198.000,2.000,nan,15.000,1.000,129.000,886.000,381.000,...,971257225.000,1.000,635.328,48.131,413.926,2842.933,1222.525,1193.647,426.761,54.548
1,Agoura Hills,20762.000,19.000,0.000,nan,2.000,1.000,7.000,306.000,109.000,...,431060644.000,0.000,91.513,9.633,33.715,1473.846,524.998,891.051,57.798,33.715
2,Alameda,76206.000,158.000,0.000,nan,10.000,1.000,63.000,1902.000,287.000,...,5807354436.000,0.000,207.333,13.122,82.671,2495.866,376.611,1686.219,433.037,22.308
3,Albany,19104.000,29.000,0.000,nan,1.000,1.000,4.000,557.000,94.000,...,364962816.000,0.000,151.801,5.235,20.938,2915.620,492.044,2030.988,392.588,36.642
4,Alhambra,84710.000,163.000,1.000,nan,9.000,1.000,72.000,1774.000,344.000,...,7175784100.000,1.000,192.421,10.624,84.996,2094.204,406.091,1411.876,276.237,8.263


In [39]:
num_columns = ['Population', 'ViolentCrime', 'Rape', 'AggravatedAssault', 'PropertyCrime', 'Burglary','LarcenyTheft',
                                               'MotorVehicleTheft', 'Arson']

In [40]:
# Acquire, load, and preview the data, for California
regr = linear_model.LinearRegression()
     
Y = df['PropertyCrime'].values.reshape(-1,1)
X = df[['LarcenyTheft', 'MotorVehicleTheft','Rape','AggravatedAssault']] # 95-99%, with 4.2183% difference
# X = df[['LarcenyTheft', 'MotorVehicleTheft','Rape','AggravatedAssault','Burglary']] # 100% and delta=0%!!!


# X = df[['LarcenyTheft']]


regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

# Let's run with the holdouts, and run a cross-validation
print("\nHere comes the holdouts and cross-validation.\n")
func=regr
run_with_the_holdouts(func, X, Y, 0.20)
cv_score=cross_val_score(func, X, Y, cv=10)
print(cv_score)
return_diff_min_max_cv_score(cv_score)


Coefficients: 
 [[ 1.05703911  1.42198015 -2.52731167  1.07970083]]

Intercept: 
 [89.54205455]

R-squared:
0.9984353206317621

Here comes the holdouts and cross-validation.

With 20% Holdout: 0.9942613283279509
Testing on Sample: 0.9984353206317621
[0.98904069 0.98158867 0.9860512  0.99497874 0.98710934 0.99977376
 0.99274324 0.99458641 0.99533938 0.95306652]
min cv_score=95.3067%, max=99.9774%, mean=98.7428%, and delta is 4.6707%


In [41]:
# Acquire, load, and preview the data, the old model, from Unit 2 - Lesson 4 - Project 4, New York
print("The old model, from New York, using the California data")
regr = linear_model.LinearRegression()

Y = df_pop['PropertyCrime'].values.reshape(-1,1)
X = df_pop[['LarcenyTheft', 'MotorVehicleTheft']]

# X = df[['LarcenyTheft']]


regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

# Let's run with the holdouts, and run a cross-validation
print("\nHere comes the holdouts and cross-validation.\n")
func=regr
run_with_the_holdouts(func, X, Y, 0.20)
cv_score=cross_val_score(func, X, Y, cv=10)
print(cv_score)
return_diff_min_max_cv_score(cv_score)

The old model, from New York, using the California data

Coefficients: 
 [[1.1094172  1.64125383]]

Intercept: 
 [90.32126898]

R-squared:
0.9979072868934545

Here comes the holdouts and cross-validation.

With 20% Holdout: 0.9952798466362992
Testing on Sample: 0.9979072868934545
[0.9906775  0.99219522 0.98561321 0.99626416 0.98556972 0.9996921
 0.99561699 0.99706365 0.98450105 0.9456844 ]
min cv_score=94.5684%, max=99.9692%, mean=98.7288%, and delta is 5.4008%


### Writeup  
Using the model from this project yielded great results 99% for both the new and old models, using the California data.  Finding the right independent variables, and making them few, brings about great results.  Both models yielded 5% difference between the old and new holdouts.